# Model
当今主流大模型从架构上大致可分为稠密（Dense）模型和混合专家模型（Mixture of Expert, MoE）模型。

稠密模型中所有参数在每次计算时都会参与运算；混合专家模型则将不同的“专家”模块组合，根据输入选择合适的专家处理，能在保证效果的同时减少计算量和参数量。

MiniMind 模型在Llama 3.1 的基础上设计，基于经典的Transformer Decoder-Only 架构。

## MiniMind Dense Model

In [3]:
import math
import struct
import inspect
import time
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from model.LMConfig import LMConfig
from typing import Any, Optional, Tuple, List
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast

### 均方根层归一化（RMSNorm）
RMSNorm 是对 LayerNorm 的改进，移除了均值项，可以视为 LayerNorm在均值为 0 时的特例。
* LayerNorm
$$
y = \frac{x- E(x)} {\sqrt {Var(x)+\epsilon}} * \gamma + \beta
$$
* RMS Norm
$$
a_i = \frac {a_i}{RMS(a)+\epsilon}*\gamma, \ where \ RMS(a) = \sqrt { \frac {1} {n} \sum_{i=1}^{n} a^2_i}
$$
RMS Norm在Layer Norm的基础上舍弃了中心化操作，仅用缩放进行归一化，其不改变数据原本的分布，有利于激活函数输出的稳定

In [4]:
class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-5):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        # x: [batch_size, seq_len, dim]
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
    
    def forward(self, x):
        return self.weight * self._norm(x.float()).type_as(x)  # 输出与输入的数据类型一致，避免后续计算破坏混合精度训练

## Rotary Position Embedding(RoPE) 旋转位置编码
### Rotary Position Embedding, RoPE

旋转位置编码是一种能将相对位置信息集成到 self-attention 中, 进而提升 transformer 架构性能的位置编码方式, 和绝对位置编码相比, RoPE 具有很好的外推性, 是目前的主流位置编码方式.

外推性的解释, 通俗来说就是训练的时候限制了 512 的上下文长度，那么推理时如果面对超过该长度的文本，LLM 可能无法正确处理.

- **绝对位置编码**

绝对位置编码是早期 Transformer 架构采用的绝对位置编码方案，及那个每个位置映射为固定的向量表示.

$$f_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i,i)=\boldsymbol{W}_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i+\boldsymbol{p}_i)$$

其中编码向量 $p_i$ 的计算使用如下公式：

$$\boldsymbol{p}_{i,2t}=\sin\left(k/1000^{2t/d}\right), \boldsymbol{p}_{i,2t+1}=\cos\left(k/1000^{2t/d}\right)$$

正如其名，绝对位置编码只考虑了输入序列中的绝对位置关系，对于 token 之间的相对信息则没有纳入考虑.

- **旋转位置编码**

假定 query 和 key 的内积操作可以被函数 g 表示，该函数 g 的输入是词嵌入向量 $x_m, x_n$ 和它们之间的相对位置 $m-n$:

$$<f_q(x_m ,m), f_k(x_n, n)>=g(x_m, x_n, m, n)$$

旋转位置编码就是找到一个使上式成立的位置编码方式. 

出于认识的目的，我们省略复杂的数学推导，直接看 RoPE 的的结论：

存在这样一个正交矩阵：

$$\boldsymbol{R}_{\Theta,m}^d=\underbrace{\begin{pmatrix}\cos m\theta_0&-\sin m\theta_0&0&0&\cdots&0&0\\\sin m\theta_0&\cos m\theta_0&0&0&\cdots&0&0\\0&0&\cos m\theta_1&-\sin m\theta_1&\cdots&0&0\\0&0&\sin m\theta_1&\cos m\theta_1&\cdots&0&0\\\vdots&\vdots&\vdots&\vdots&\ddots&\vdots&\vdots\\0&0&0&0&\cdots&\cos m\theta_{d/2-1}&-\sin m\theta_{d/2-1}&-\sin m\theta_{d/2-1}\end{pmatrix}}_{\boldsymbol{W}_m}$$

其中，$\Theta=\left\{\theta_i=10000^{-2(i-1)/d},i\in[1,2,\ldots,d/2]\right\}$

我们可以将 query 和 key 的内积操作转换为与原始向量 $x$ 相关的以下等价形式：

$$
\boldsymbol{q}_m^\mathbf{T}\boldsymbol{k}_n=\left(\boldsymbol{R}_{\Theta,m}^d\boldsymbol{W}_q\boldsymbol{x}_m\right)^\mathbf{T}\left(\boldsymbol{R}_{\Theta,n}^d\boldsymbol{W}_k\boldsymbol{x}_n\right)=\boldsymbol{x}_m^\mathbf{T}\boldsymbol{W}_q\boldsymbol{R}_{\Theta,n-m}^d\boldsymbol{W}_k\boldsymbol{x}_n
$$

其中， $\boldsymbol{R}_{\Theta,n-m}^d=\left(\boldsymbol{R}_{\Theta,m}^d\right)^\mathbf{T}\boldsymbol{R}_{\Theta,n}^d$.

由于 $\boldsymbol{R}_{\Theta,m}^d$ 的稀疏性，直接使用矩阵乘法会浪费算力，因此代码中采用下述方式实现：

$$\boldsymbol{R}_{\Theta,m}^{d}\boldsymbol{x}=\begin{pmatrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\\vdots\\x_{d-2}\\x_{d-1}\end{pmatrix}\otimes\begin{pmatrix}\cos m\theta_{0}\\\cos m\theta_{0}\\\cos m\theta_{1}\\\cos m\theta_{1}\\\vdots\\\cos m\theta_{d/2-1}\\\cos m\theta_{d/2-1}\end{pmatrix}+\begin{pmatrix}-x_{1}\\x_{0}\\-x_{3}\\x_{2}\\\vdots\\-x_{d-1}\\x_{d-2}\end{pmatrix}\otimes\begin{pmatrix}\sin m\theta_{0}\\\sin m\theta_{0}\\\sin m\theta_{1}\\\sin m\theta_{1}\\\vdots\\\sin m\theta_{d/2-1}\\\sin m\theta_{d/2-1}\end{pmatrix}
$$

In [11]:
# 在 RoPE 中预先计算旋转角度对应的复数（cosθ + i·sinθ）值 mθ
def precompute_freqs_cis(dim: int, end: int = int(32 * 1024), theta: float = 1e6):
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))  # \theta_i = 10000^{-2i/d}, i \in [0, 1, ..., d/2-1]
    t = torch.arange(0, end, device=freqs.device)
    freqs = torch.outer(t, freqs).float()
    # [seq_len, dim]
    freqs_cos = torch.cat([torch.cos(freqs), torch.cos(freqs)], dim=-1)
    freqs_sin = torch.cat([torch.sin(freqs), torch.sin(freqs)], dim=-1)
    return freqs_cos, freqs_sin

def apply_rotary_pos_emb(q, k, cos, sin, unsqueeze_dim=1):
    def rotate_half(x):
        # 将 x 的前半部分和后半部分（取反）进行交换，代替 sin 的取反
        return torch.cat([-x[..., x.shape[-1] // 2:], x[..., :x.shape[-1] // 2]], dim=-1)
    
    # q, k: [batch_size, seq_len, num_heads, head_dim]
    # 对 q, k 和 cos, sin 进行广播运算，需要先匹配维度
    # cos, sin [seq_len, head_dim] -> [(1), seq_len, 1, head_dim] 即对所有 batch, head 进行相同的广播运算
    q_embed = (q * cos.unsqueeze(unsqueeze_dim)) + (rotate_half(q) * sin.unsqueeze(unsqueeze_dim))
    k_embed = (k * cos.unsqueeze(unsqueeze_dim)) + (rotate_half(k) * sin.unsqueeze(unsqueeze_dim))
    return q_embed, k_embed

xq,  xk = torch.randn((2,  4,  4,  4)), torch.randn((2,  4,  4,  4)) # (batch_size,  sequence_length,  num_heads,  head_dim)
freqs_cos, freqs_sin = precompute_freqs_cis(4,  4)
q_embed, k_embed = apply_rotary_pos_emb(xq, xk, freqs_cos, freqs_sin)
q_embed


tensor([[[[-0.1271,  0.8812, -0.6388, -0.1893],
          [ 1.0094, -1.1864, -0.0300,  0.7832],
          [ 0.4957,  1.0637,  0.7216, -0.4009],
          [-0.7514, -0.3974,  1.1031, -0.1514]],

         [[ 1.4954, -0.1738,  0.0650, -2.2456],
          [ 0.4194, -0.3086, -0.1484,  0.3030],
          [ 1.0979,  1.3605,  0.9210, -0.6754],
          [ 0.0564,  0.9247, -0.4771,  0.3487]],

         [[ 0.4618, -0.9191, -0.4120,  0.6553],
          [ 1.0198, -0.0600, -0.2367, -0.4534],
          [-0.0824, -1.9700, -0.0934,  0.2286],
          [-0.1301,  0.9609,  0.0657, -0.6252]],

         [[ 1.3434, -0.8016,  1.6825,  0.1456],
          [-0.0197, -2.1288,  1.7042,  0.0105],
          [ 0.4340, -0.6796,  0.0223,  0.7759],
          [-0.5726,  0.7877,  0.4440,  1.6066]]],


        [[[ 1.5729,  0.4159, -0.1599,  0.3882],
          [-0.1739,  0.2431, -1.2901,  0.4096],
          [-0.4292,  0.4393, -0.4322, -0.7097],
          [ 0.1817,  1.2392,  0.2762,  0.7456]],

         [[-0.0240,  0.1781,

### Attention
注意力机制是 Transformer 架构的核心组件，能够有效捕捉长序列内各元素间的依赖关系，通过计算输入序列中不同位置元素间的注意力得分，对重要性进行建模

在 MiniMindLM 模型中，Attention Block 包含下面的机制和模块：
1. GQA (Group Query Attention) 分组查询注意力
2. KV Cache
3. SwiGLU
#### GQA
GQA 是对多头自注意力机制的扩展，通过对查询头分组，提高计算效率

GQA 将 h 个查询头分为 G 组，每组包含 h / G 个查询头，共享一个公共的键和值

**GQA 相比传统 MHA，减少了键和值的数量，降低了计算量和内存开销，提高了推理速度**

### KV Cache
在语言模型生成文本的过程中，每生成一个新的 token，模型都需要计算注意力得分，以确定当前位置与之前所有位置的相关性.

比如以下内容：

1. seq = [tok1] (位置 1):

   S_1 = [ (Q1 * K1^T) / sqrt(d_k) ] (只有自己和自己的分数)

   A_1 = softmax(S_1) = [1.0] (唯一选项，权重为1)

   Output_1 = 1.0 * V1 = V1

2. seq = [tok1, tok2] (计算位置 2 的输出):

   S_2 = [ (Q2 * K1^T) / sqrt(d_k), (Q2 * K2^T) / sqrt(d_k) ] (位置2对位置1和位置2的分数)

   A_2 = softmax(S_2) = [a_21, a_22] (对这两个分数进行归一化，a_21 + a_22 = 1)

   Output_2 = a_21 * V1 + a_22 * V2

3. seq = [tok1, tok2, tok3] (计算位置 3 的输出):

   S_3 = [ (Q3 * K1^T) / sqrt(d_k), (Q3 * K2^T) / sqrt(d_k), (Q3 * K3^T) / sqrt(d_k) ] (位置3对位置1,2,3的分数)

   A_3 = softmax(S_3) = [a_31, a_32, a_33] (对这三个分数进行归一化，a_31 + a_32 + a_33 = 1)
   
   Output_3 = a_31 * V1 + a_32 * V2 + a_33 * V3

不难发现，大模型生成一个 token 后的注意力计算中，总会用到 token 序列的历史 KV 值，导致重复计算，KV Cache 的设计正是为了通过缓存历史 KV 值，节省计算开销.

KV Cache 能够有效压缩大模型推理时的显存占用.

注意力机制是在**计算某个位置的输出时，对该位置与所有可见位置（已生成的位置）的注意力分数进行一次性的 `softmax` 归一化**。

#### 📌 总结与关键点

1.  **`softmax` per Row (per Query)：** `softmax` 操作是针对**一个特定查询位置 `i`** 的所有（未被掩码的）键位置的分数进行归一化。它发生在计算该查询位置 `i` 的输出向量之前。
2.  **注意力权重的意义：** 归一化后的注意力权重 `a_ij` 代表了 **位置 `i`（查询）对位置 `j`（键值）的“关注程度”**。所有权重之和为 1。
3.  **输出是加权和：** 位置 `i` 的输出是其对所有可见位置 `j` 的值向量 `V_j` 的加权和，权重就是 `a_ij`。
4.  **自回归生成中的缓存 (KV Cache)：** 在像GPT这样的Decoder-only模型进行自回归生成时（预测下一个token）：
    *   当生成第 `i` 个 token 时，我们只需要计算**当前**的 `Q_i`。
    *   所有之前位置 `j < i` 的 `K_j` 和 `V_j` 已经从之前的步骤中计算并**缓存**好了 (这就是著名的 **KV Cache**)。
    *   因此，计算 `Output_i` 只需要：
        *   计算 `Q_i`；
        *   用 `Q_i` 和缓存的 `K_{1:i}` 计算分数 `S_i`；
        *   对 `S_i` 做 `softmax` 得到 `A_i`；
        *   用 `A_i` 和缓存的 `V_{1:i}` 计算加权和 `Output_i`。
    *   计算完 `Output_i` 后，我们会计算并缓存**当前**位置的 `K_i` 和 `V_i`，供后续生成步骤使用。
  
### SwiGLU
SwiGLU 是一种激活函数变体:
$$
SwiGLU(x, W, V, b, c) = Swish(xW+b) \otimes (xV+c)
$$
其中 $Swish(x)=x \cdot \sigma (\beta x)$

与传统的 ReLU 激活函数相比，SwiGLU 具有更好的平滑性和非线性表达能力，由于其门控机制，在处理信息筛选和流动方面有独特的优势

In [13]:
from transformers import PretrainedConfig

class MiniMindConfig(PretrainedConfig):
    model_type = "minimind"
    
    def __init__(
        self,
        dropout: float = 0.0,
        bos_token_id: int = 1,
        eos_token_id: int = 2,
        hidden_act: str = 'silu',
        hidden_size: int = 512,
        intermediate_size: int = None,
        max_position_embeddings: int = 32768,
        num_attention_heads: int = 8,
        num_hidden_layers: int = 8,
        num_key_value_heads: int = 2,
        vocab_size: int = 6400,
        rms_norm_eps: float = 1e-5,
        rope_theta: int = 1000000,
        flash_attn: bool = True,
    ):
        super().__init__()
        self.dropout = dropout
        self.bos_token_id = bos_token_id
        self.eos_token_id = eos_token_id
        self.hidden_act = hidden_act
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.max_position_embeddings = max_position_embeddings
        self.num_attention_heads = num_attention_heads
        self.num_hidden_layers = num_hidden_layers
        self.num_key_value_heads = num_key_value_heads
        self.vocab_size = vocab_size
        self.rms_norm_eps = rms_norm_eps
        self.rope_theta = rope_theta
        self.flash_attn = flash_attn
        
        

def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    """使 KV 头数适应 Query 头数， 执行矩阵乘法并行运算
    等价于 torch.repeat_interleave(x, dim=2, repeats=n_rep)"""
    batch_size, seq_len, num_kv_heads, head_dim = x.shape
    if n_rep == 1:
        return x
    return (
        x[:, :, :, None, :]  # 等价于 x.unsqueeze(3)
        .expand(batch_size, seq_len, num_kv_heads, n_rep, head_dim)
        .reshape(batch_size, seq_len, num_kv_heads * n_rep, head_dim)
    )
    
class Attention(nn.Module):
    def __init__(self, args: MiniMindConfig):
        super().__init__()
        self.num_key_value_heads = args.num_attention_heads if args.num_key_value_heads is None else args.num_key_value_heads
        assert args.num_attention_heads % self.num_key_value_heads == 0
        self.n_local_heads = args.num_attention_heads
        self.n_local_kv_heads = args.num_key_value_heads
        self.n_rep = self.n_local_heads // self.n_local_kv_heads
        self.head_dim = args.hidden_size // args.num_attention_heads  # query 头映射的 head_dim
        self.q_proj = nn.Linear(args.hidden_size, args.num_attention_heads * self.head_dim, bias=False)
        self.k_proj = nn.Linear(args.hidden_size, args.num_key_value_heads * self.head_dim, bias=False)
        self.v_proj = nn.Linear(args.hidden_size, args.num_key_value_heads * self.head_dim, bias=False)
        self.o_proj = nn.Linear(args.num_attention_heads * self.head_dim, args.hidden_size, bias=False)
        self.attn_dropout = nn.Dropout(args.dropout)
        self.resid_dropout = nn.Dropout(args.dropout)
        self.dropout = args.dropout
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention') and args.flash_attn
        
    def forward(self,
                x: torch.Tensor,
                position_embeddings: Tuple[torch.Tensor, torch.Tensor],  # 接收 cos 和 sin
                past_key_value: Optional[Tuple[torch.Tensor, torch.Tensor]] = None,
                use_cache=False,
                attention_mask: Optional[torch.Tensor] = None,):
        batch_size, seq_len, _ = x.shape
        ############## Forward QKV & RoPE ##############
        xq, xk, xv = self.q_proj(x), self.k_proj(x), self.v_proj(x)
        xq = xq.view(batch_size, seq_len, self.n_local_heads, self.head_dim)
        xk = xk.view(batch_size, seq_len, self.n_local_kv_heads, self.head_dim)
        xv = xv.view(batch_size, seq_len, self.n_local_kv_heads, self.head_dim)
        
        cos, sin = position_embeddings
        xq, xk = apply_rotary_pos_emb(xq, xk, cos[:seq_len], sin[:seq_len])  # 截断至 seq_len
        
        # kv_cache 实现
        if past_key_value is not None:
            xk = torch.cat([past_key_value[0], xk], dim=1)  # 缓存每一个 token 的 k, v
            xv = torch.cat([past_key_value[1], xk], dim=1)
        past_kv = (xk, xv) if use_cache else None
        
        # [batch_size, seq_len, num_heads, head_dim] -> [bsz, num_heads, seq_len, head_dim]
        xq, xk, xv = (
            xq.transpose(1, 2),
            repeat_kv(xk, self.n_rep).transpose(1, 2),
            repeat_kv(xv, self.n_rep).transpose(1, 2),
        )
        
        ############ Scaled Dot Production #############
        if self.flash and seq_len != 1:
            dropout_p = self.dropout if self.training else 0.0
            attn_mask = None  # 这里的 attention_mask 指的是 padding 的掩码
            if attention_mask is not None:
                attn_mask = attention_mask.view(batch_size, 1, 1, -1).expand(batch_size, self.n_local_heads, seq_len, -1)  # attention_mask 形状为 [bsz, seq_len] 扩展后形状为 [bsz, n_heads, seq_len, seq_len]
                attn_mask = attn_mask.bool()
            output = F.scaled_dot_product_attention(xq, xk, xv, attn_mask=attn_mask, dropout_p=dropout_p, is_causal=True)
        else:
            # 普通注意力机制
            scores = (xq @ xk.transpose(-2, -1)) / math.sqrt(self.head_dim)  # 缩放点积
            scores = scores + torch.triu(
                torch.full((1, 1, seq_len, seq_len), float("inf"), device=scores.device),
                diagonal=1
            )
            
            # 处理 padding 的掩码
            if attention_mask is not None:
                extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)  # [batch_size, 1, 1, seq_len]
                extended_attention_mask = (1.0 - extended_attention_mask) * -1e9  # padding 的部分变为 -inf
                scores += extended_attention_mask
                
            scores = F.softmax(scores.float(), dim=-1).type_as(xq)
            scores = self.attn_dropout(scores)
            output = scores @ xv  # [..., seq_len, seq_len] @ [..., seq_len, head_dim] -> [..., seq_len, head_dim]
        
        output = output.transpose(1, 2).reshape(batch_size, seq_len, -1)  # -> [batch_size, seq_len, dim] 等价于将所有头的输出维度拼接
        output = self.resid_dropout(self.o_proj(output))
        return output, past_kv

In [25]:
attn = Attention(MiniMindConfig())
x = torch.randn((2, 4, 512))
cos, sin = precompute_freqs_cis(64, 4)
output, past_kv = attn(x, (cos, sin), use_cache=True)
print(f'输入张量 x ：size = {x.shape}，RoPE 旋转角： size = {cos.shape}')
print(f'输出 output: size = {output.shape},  kv_cache 基本信息：size_key = {past_kv[0].shape}, size_value = {past_kv[1].shape}')

输入张量 x ：size = torch.Size([2, 4, 512])，RoPE 旋转角： size = torch.Size([4, 64])
输出 output: size = torch.Size([2, 4, 512]),  kv_cache 基本信息：size_key = torch.Size([2, 4, 2, 64]), size_value = torch.Size([2, 4, 2, 64])


In [16]:
output

tensor([[[-0.4768,  0.0344,  0.0520,  ..., -0.0625,  0.4398, -0.1725],
         [-0.3904, -0.0107,  0.0797,  ..., -0.1729,  0.1587, -0.2356],
         [-0.1888,  0.0010,  0.2464,  ..., -0.1216,  0.3145, -0.0996],
         [-0.0146,  0.0089,  0.1481,  ..., -0.1093,  0.2050,  0.0232]],

        [[-0.4153, -0.3545,  0.0203,  ...,  0.1221, -0.1954, -0.2919],
         [-0.0517, -0.3404, -0.2075,  ...,  0.0648,  0.1257, -0.0926],
         [ 0.1259, -0.2642, -0.1286,  ...,  0.1233, -0.0690, -0.1381],
         [ 0.0247, -0.2238, -0.3681,  ...,  0.0936, -0.1268, -0.0488]]],
       grad_fn=<UnsafeViewBackward0>)